In [1]:
import torch
import numpy as np
import pandas as pd
import copy
import torch.nn.functional as F
import scipy.sparse as sp
from torch_geometric.nn import GCNConv,GATConv,SAGEConv
from torch_geometric.datasets import Planetoid
from torch.nn import Linear
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import datetime
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
torch.cuda.set_device(1)
torch.cuda.current_device()

Using cuda device


1

In [3]:
from torch.nn import Linear
from torch_geometric.nn import GraphConv

class GNN(torch.nn.Module):
    def __init__(self, num_node_features,hidden_channels,num_classes):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(num_node_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels,num_classes)
        
    def forward(self, x, edge_index, batch,edge_weight=None):
        # 1. 获得节点嵌入
        x = self.conv1(x, edge_index,edge_weight)
        x = x.relu()
        x = self.conv2(x, edge_index,edge_weight)
        x = x.relu()
        x = self.conv3(x, edge_index,edge_weight)
        
        # 2. Readout layer
        x = global_mean_pool(x, batch)   # [batch_size, hidden_channels]
        
        # 3. 分类器
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x

In [20]:
#model_tuning = GNN(num_node_features=1,hidden_channels=64,num_classes=31).to(device)
#checkpoint = torch.load('/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reaction.PCA_convEdgeWeight.1018.pt')
model_tuning = torch.load("/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reaction.PCA_convEdgeWeight.1018.pt",map_location=device)
#model_tuning.load_state_dict(checkpoint)
model_tuning.lin = Linear(64,20)
print (model_tuning)
model_tuning = model_tuning.to(device)
# 固定除线性层外的layer 参数
for name,layer in model_tuning.named_parameters():
    if (name == 'lin.weight' or (name == 'lin.bias')):
        #print (name,layer)
        layer.requires_grad = True
    else:
        layer.requires_grad = False

GNN(
  (conv1): GraphConv(1, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=20, bias=True)
)


In [ ]:
for name, param in model_tuning.named_parameters(): print(name, param)

In [5]:
## 读取TCGA 样本对应的组织标签
TCGA_sample_label = pd.read_csv('/public/home/liujunwu/workdir/scripts/GNN_Reactome/TCGA_exp/GSEA/TCGA_sample.label',header=None,sep=",")
TCGA_sample_label.columns = ['Sample','short']
TCGA_tissue_label = pd.read_csv('/public/home/liujunwu/workdir/scripts/GNN_Reactome/TCGA_exp/GSEA/TCGA_tissue.label',header=None,sep='\t')
TCGA_tissue_label.columns = ['short','long']
TCGA_merge = pd.merge(left=TCGA_sample_label, 
                   right=TCGA_tissue_label, 
                   how='left', 
                   on='short')
#print (TCGA_merge.iloc[0:3,0:3])
TCGA_sample_label_dict = TCGA_merge[['Sample', "long"]].set_index("Sample").to_dict(orient='dict')["long"]
#print (TCGA_sample_label_dict["TCGA-BT-A20R-11A-11R-A16R-07"])
TCGAsample_full_labels = sorted(list(set(TCGA_merge["long"].tolist()))) ## 唯一值
TCGAsample_full_labels_dict = dict(zip(TCGAsample_full_labels,range(len(TCGAsample_full_labels)))) ## label对应的index
TCGAsample_full_labels_rev_dict = dict(zip(range(len(TCGAsample_full_labels)),TCGAsample_full_labels)) ## label对应的index
TCGA_list_labels = []
print (TCGAsample_full_labels_dict)
for x in range(len(TCGAsample_full_labels)):
    TCGA_list_labels.append([x])
TCGA_list_labels_tensor = torch.tensor(TCGA_list_labels,dtype=torch.int64)
#print (TCGA_list_labels[TCGAsample_full_labels_dict[TCGA_sample_label_dict["TCGA-BT-A20R-11A-11R-A16R-07"]]])

{'Bile duct': 0, 'Bladder': 1, 'Brain': 2, 'Breast': 3, 'Cervix Uteri': 4, 'Colorectal': 5, 'Esophagus': 6, 'Head and neck': 7, 'Kidney': 8, 'Liver': 9, 'Lung': 10, 'Muscle': 11, 'Nerve': 12, 'Pancreas': 13, 'Prostate': 14, 'Skin': 15, 'Stomach': 16, 'Thymus': 17, 'Thyroid': 18, 'Uterus': 19}


In [6]:
## 读取reaction包含的基因
ReactionGeneFile = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reactome_reaction.ProteinReactionNodes.txt"
ReactionGene = pd.read_csv(ReactionGeneFile,header=None,sep='\t')
ReactionGene.columns = ["Reaction","Gene"]
ReactionGene_dict = dict(zip(ReactionGene["Reaction"],ReactionGene["Gene"])) ## Reaction 对应的Gene
print (ReactionGene_dict['R-HSA-1112666'])
del ReactionGene

IGHV3-11,IGKV1D-33,SYK,SH3KBP1,IGHV1-69,IGLV1-47,IGKC,GRB2,IGHV2-70,IGLC7,IGKV3-11,IGHM,IGLV1-51,IGLV1-44,IGKV3-20,IGKV2D-28,IGKV2-30,NCK1,IGHV2-5,IGLC3,IGKV1-12,IGLV6-57,IGLV1-40,IGHD,IGKV2D-40,SOS1,IGLV3-25,IGHV3-23,IGHV1-2,IGLV2-14,IGKV2-28,IGKV1D-16,IGLV3-19,IGHV4-59,IGLV2-23,IGKV1D-12,IGHV3-33,IGLC1,BLNK,PLCG2,IGKV2D-30,IGLV3-27,IGKV3-15,IGLV2-11,IGHV3-48,IGHV3-9,BTK,IGLV2-8,IGKV1-17,IGKV3D-20,IGKV1-39,IGHV3-13,IGKV1-16,IGKV1D-39,IGHV4-34,IGKV2-29,IGLV7-43,IGLC6,IGKV4-1,IGLV3-1,IGKV5-2,CD79A,IGHV1-46,IGLV3-21,IGKV1-33,CD79B,IGKV1-5,IGHV4-39,VAV1,IGHV3-7,IGLC2,IGHV3-53,IGHV3-30


In [7]:
edges_file = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reactome_reaction.edges.txt"
edges = pd.read_csv(edges_file,header=None,sep="\t")
edges.columns = ["edge1","edge2","edge1_type","edge2_type"] 
uniq_output_nodes = sorted(list(set(edges["edge2"].tolist()))) ## 取得唯一入度的点
uniq_nodes = sorted(list(set(edges["edge1"].tolist()) | set(edges["edge2"].tolist())))

In [8]:
edges = torch.load("/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/Reactome_reaction_edges.pt")
print (edges)

tensor([[11573, 11462,     3,  ...,  6095,  9849,  9046],
        [    0,     1,     2,  ..., 11713, 11713, 11713]])


In [9]:
TCGA_expfile = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/TCGA_exp/all_TCGA.exp.csv" ## 小数据集测试
#GTEx_expfile = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/GTEx_exp_down5000.csv" ## 小数据集测试
TCGA_exp = pd.read_csv(TCGA_expfile,header=0,sep=",")
TCGA_genelist = TCGA_exp.columns

In [10]:
sample_full_labels = sorted(list(set(TCGA_exp["Label"].tolist())))
sample_full_labels_dict = dict(zip(sample_full_labels,range(len(sample_full_labels)))) ## label对应的index
sample_index_dict = dict(zip(TCGA_exp["Sample"],TCGA_exp.index)) 
list_labels = []
print (sample_full_labels_dict)
sample_dict = dict(zip(TCGA_exp["Sample"],TCGA_exp["Label"]))
print (sample_full_labels_dict[sample_dict["TCGA-A7-A13G-11A-51R-A13Q-07"]])
for x in range(len(sample_full_labels)):
    list_labels.append([x])
list_labels_tensor = torch.tensor(list_labels,dtype=torch.int64)
print (list_labels_tensor[sample_full_labels_dict[sample_dict["TCGA-A7-A13G-11A-51R-A13Q-07"]]])

{'BLCA': 0, 'BRCA': 1, 'CESC': 2, 'CHOL': 3, 'COAD': 4, 'ESCA': 5, 'GBM': 6, 'HNSC': 7, 'KICH': 8, 'KIRC': 9, 'KIRP': 10, 'LIHC': 11, 'LUAD': 12, 'LUSC': 13, 'PAAD': 14, 'PCPG': 15, 'PRAD': 16, 'READ': 17, 'SARC': 18, 'SKCM': 19, 'STAD': 20, 'THCA': 21, 'THYM': 22, 'UCEC': 23}
1
tensor([1])


In [14]:
TCGA_uniq_labels = TCGA_exp["Label"].unique()
print (TCGA_uniq_labels)

['BLCA' 'BRCA' 'CESC' 'CHOL' 'COAD' 'ESCA' 'GBM' 'HNSC' 'KICH' 'KIRC'
 'KIRP' 'LIHC' 'LUAD' 'LUSC' 'PAAD' 'PCPG' 'PRAD' 'READ' 'SARC' 'SKCM'
 'STAD' 'THCA' 'THYM' 'UCEC']


In [ ]:
edges

In [12]:
## 对于每一个reaction , PCA 得到的主成分 PCA1值
import glob
import re
import os
import math
from sklearn import preprocessing

## 对于每一个reaction , PCA 得到的主成分 PCA1值
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(n_components=2)
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
sample_reaction_pca = pd.DataFrame()
n = 0
reaction_PCA_values = []
for node in iter(uniq_nodes): ## 计算所有reaction节点 对应的PCA1 values
    #print (node)
    #node = "R-HSA-1006169"
    if (node in ReactionGene_dict.keys()):
        genelist = ReactionGene_dict[node].split(",")
        common_gene = list(set(genelist) & set(TCGA_genelist))
        ## 去除bulk中不存在的基因
        sub_exp = TCGA_exp.loc[:,common_gene]
        #print (sub_exp)
        if (sub_exp.shape[1] >= 2):
            #sub_exp = sub_exp.values
            sub_exp = StandardScaler().fit_transform(sub_exp.values)
            principalComponents = pca.fit_transform(sub_exp)
            PCA_values =  pd.DataFrame(data = principalComponents,columns = ["PCA1","PCA2"])
            node_PCA_values = PCA_values["PCA1"]
        elif (sub_exp.shape[1] == 1): ## 如果reaction中只包含一个基因，无法做PCA，则对该基因表达量进行标准化
            sub_exp = StandardScaler().fit_transform(sub_exp.values)
            node_PCA_values = pd.DataFrame(data = sub_exp, columns=["PCA1"])
            #print (node_PCA_values)
        else:
            node_PCA_values = pd.DataFrame(np.zeros((TCGA_exp.shape[0],1)),columns=["PCA1"])
    else:
        node_PCA_values = pd.DataFrame(np.zeros((TCGA_exp.shape[0],1)),columns=["PCA1"])
    reaction_PCA_values.append(node_PCA_values)
    #sample_reaction_pca = pd.concat([sample_reaction_pca,node_PCA_values],axis=1,join='outer',ignore_index=True)
    #n += 1
    #if n >3:break
    #break
sample_reaction_pca = pd.concat(reaction_PCA_values,axis=1,join='outer',ignore_index=True)
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
print (sample_reaction_pca.iloc[0:10,0:10])
print (sample_reaction_pca.shape)


2023-10-18  16:02:42
2023-10-18  16:03:42
          0         1         2         3         4         5         6  \
0 -0.680701  1.081983  3.583182  3.583182 -0.426539 -0.327598 -0.339868   
1  1.447236 -0.260758  3.589668  3.589668  0.537294  0.264704 -1.489758   
2 -0.058676 -0.586946 -0.863641 -0.863641 -1.373529 -3.236777  2.921103   
3  1.311051  0.148850  0.625590  0.625590  0.459675 -0.155701 -1.049457   
4 -0.812770  0.956028  3.280487  3.280487  0.197606 -0.939456 -0.070037   
5  1.700964  0.720706 -0.206392 -0.206392  0.635212  1.227438 -1.819872   
6 -0.491500  1.055944  2.225275  2.225275 -0.103181  0.665602 -1.316430   
7  0.576235 -0.161400  1.168823  1.168823 -1.917876 -3.758887  2.982174   
8 -0.328893  0.760097  1.740886  1.740886 -0.856205 -1.337907  0.608190   
9  1.292049  0.754481  1.939269  1.939269 -0.108517  0.989916 -1.459466   

          7         8         9  
0 -0.327598 -0.427571  0.547441  
1  0.264704 -0.424239 -1.006817  
2 -3.236777  1.560242 -0.69904

In [ ]:
sample_reaction_pca.loc[0]

In [15]:
## 根据TCGA不同组织，选取一半样本作为微调的训练集 
TCGA_tuning_sets = []
TCGA_test_sets = []
for tissue in TCGA_uniq_labels:
    TCGA_tissue_sample = TCGA_exp[TCGA_exp["Label"] == tissue]["Sample"]
    tuning_sample = TCGA_tissue_sample.sample(math.ceil(TCGA_tissue_sample.shape[0]/2)) ## 一半且向上取整
    tuning_sample_index = tuning_sample.index
    print (tissue,len(tuning_sample))
    for m in tuning_sample:
        sampleName = m
        sample_index = sample_index_dict[sampleName]
        sample_value = sample_reaction_pca.loc[sample_index].values
        sample_value = sample_value.reshape(sample_value.shape[0],1)
        sample_node_feature = Data(x=torch.tensor(sample_value,dtype = torch.float32),y=torch.tensor(TCGAsample_full_labels_dict[TCGA_sample_label_dict[sampleName]]),edge_index = edges)
        TCGA_tuning_sets.append(sample_node_feature)
        
    test_sample = pd.concat([TCGA_tissue_sample,tuning_sample,tuning_sample]).drop_duplicates(keep=False)
    for n in test_sample:
        sampleName = n
        sample_index = sample_index_dict[sampleName]
        sample_value = sample_reaction_pca.loc[sample_index].values
        sample_value = sample_value.reshape(sample_value.shape[0],1)
        sample_node_feature = Data(x=torch.tensor(sample_value,dtype = torch.float32),y=torch.tensor(TCGAsample_full_labels_dict[TCGA_sample_label_dict[sampleName]]),edge_index = edges)
        TCGA_test_sets.append(sample_node_feature)
        
print (len(TCGA_tuning_sets))
print (len(TCGA_test_sets))

BLCA 10
BRCA 57
CESC 2
CHOL 5
COAD 21
ESCA 6
GBM 3
HNSC 22
KICH 12
KIRC 36
KIRP 16
LIHC 25
LUAD 30
LUSC 25
PAAD 2
PCPG 2
PRAD 26
READ 5
SARC 1
SKCM 1
STAD 16
THCA 29
THYM 1
UCEC 18
371
359


In [16]:

def train_tuning():
    model_tuning.train() ## 
    for data in TCGA_train_loader:
        #print (data.y)
        data = data.to(device)
        optimizer_tuning.zero_grad()
        out = model_tuning(data.x, data.edge_index,data.batch)
        loss = criterion_tuning(out, data.y)
        loss.backward()
        optimizer_tuning.step()
def test_tuning(loader):
    model_tuning.eval()
    correct = 0
    for data in loader:                           # 批遍历测试集数据集。
        data = data.to(device)
        out = model_tuning(data.x, data.edge_index, data.batch) # 一次前向传播
        pred = out.argmax(dim=1)   # 使用概率最高的类别
        #print (pred,data.y)
        correct += int((pred == data.y).sum())           # 检查真实标签
    return correct / len(loader.dataset)


In [21]:
TCGA_train_loader =  DataLoader(TCGA_tuning_sets, batch_size=5,shuffle=True)
TCGA_test_loader = DataLoader(TCGA_test_sets, batch_size=5,shuffle=True)
optimizer_tuning = torch.optim.Adam(model_tuning.parameters(), lr=1e-4)
criterion_tuning = torch.nn.CrossEntropyLoss()
#model_tuning.train()
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))

from tqdm import *
for epoch in range(1, 401):
    train_tuning()
    #print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
    train_acc = test_tuning(TCGA_train_loader)
    #test_acc = test_tuning(TCGA_test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')

2023-10-18  20:52:16
Epoch: 001, Train Acc: 0.0701
Epoch: 002, Train Acc: 0.0916
Epoch: 003, Train Acc: 0.1348
Epoch: 004, Train Acc: 0.2210
Epoch: 005, Train Acc: 0.2722
Epoch: 006, Train Acc: 0.3046
Epoch: 007, Train Acc: 0.3235
Epoch: 008, Train Acc: 0.3504
Epoch: 009, Train Acc: 0.3639
Epoch: 010, Train Acc: 0.3747
Epoch: 011, Train Acc: 0.3935
Epoch: 012, Train Acc: 0.3989
Epoch: 013, Train Acc: 0.4043
Epoch: 014, Train Acc: 0.4124
Epoch: 015, Train Acc: 0.4394
Epoch: 016, Train Acc: 0.4528
Epoch: 017, Train Acc: 0.4690
Epoch: 018, Train Acc: 0.4933
Epoch: 019, Train Acc: 0.5067
Epoch: 020, Train Acc: 0.5202
Epoch: 021, Train Acc: 0.5310
Epoch: 022, Train Acc: 0.5391
Epoch: 023, Train Acc: 0.5499
Epoch: 024, Train Acc: 0.5580
Epoch: 025, Train Acc: 0.5660
Epoch: 026, Train Acc: 0.5795
Epoch: 027, Train Acc: 0.5768
Epoch: 028, Train Acc: 0.5822
Epoch: 029, Train Acc: 0.5903
Epoch: 030, Train Acc: 0.5984
Epoch: 031, Train Acc: 0.6011
Epoch: 032, Train Acc: 0.6092
Epoch: 033, Train A

Epoch: 274, Train Acc: 0.8059
Epoch: 275, Train Acc: 0.8059
Epoch: 276, Train Acc: 0.8032
Epoch: 277, Train Acc: 0.8032
Epoch: 278, Train Acc: 0.8032
Epoch: 279, Train Acc: 0.8032
Epoch: 280, Train Acc: 0.8032
Epoch: 281, Train Acc: 0.8032
Epoch: 282, Train Acc: 0.8032
Epoch: 283, Train Acc: 0.8059
Epoch: 284, Train Acc: 0.8032
Epoch: 285, Train Acc: 0.8032
Epoch: 286, Train Acc: 0.8059
Epoch: 287, Train Acc: 0.8059
Epoch: 288, Train Acc: 0.8059
Epoch: 289, Train Acc: 0.8059
Epoch: 290, Train Acc: 0.8059
Epoch: 291, Train Acc: 0.8032
Epoch: 292, Train Acc: 0.8032
Epoch: 293, Train Acc: 0.8059
Epoch: 294, Train Acc: 0.8059
Epoch: 295, Train Acc: 0.8059
Epoch: 296, Train Acc: 0.8059
Epoch: 297, Train Acc: 0.8059
Epoch: 298, Train Acc: 0.8059
Epoch: 299, Train Acc: 0.8059
Epoch: 300, Train Acc: 0.8059
Epoch: 301, Train Acc: 0.8032
Epoch: 302, Train Acc: 0.8032
Epoch: 303, Train Acc: 0.8032
Epoch: 304, Train Acc: 0.8032
Epoch: 305, Train Acc: 0.8032
Epoch: 306, Train Acc: 0.8032
Epoch: 307

In [22]:
def test_tuning_subclass(loader):
    model_tuning.eval()
    correct = 0
    total = 0
    label_correct = {}
    total_label = {}
    for k in range(20):
        #print (type(k))
        total_label[k] = 0
        label_correct[k] = 0
    #print (label_correct[10])
    for data in loader:# 批遍历测试集数据集。
        data = data.to(device)
        out = model_tuning(data.x, data.edge_index, data.batch) # 一次前向传播
        pred = out.argmax(dim=1)   # 使用概率最高的类别
        total += int((pred == data.y).sum())
        index = 0
        for i in pred:
            num = i.item()
            #print (num)
            #print (data.y[0].item())
            total_label[data.y[index].item()] += 1
            if (num == data.y[index].item()):
                label_correct[num] += 1
            index += 1
    for j in total_label.keys():
        correct_num = label_correct[j]
        ratio = correct_num / total_label[j] if total_label[j] != 0 else 0
        print (TCGAsample_full_labels_rev_dict[j],total_label[j],ratio)
    return total / len(loader.dataset)

In [19]:
test_acc = test_tuning_subclass(TCGA_test_loader)
#print (test_acc.)
print(f'Test Acc: {test_acc:.4f}')

Bile duct 4 0.0
Bladder 9 0.1111111111111111
Brain 2 1.0
Breast 56 0.9464285714285714
Cervix Uteri 1 0.0
Colorectal 25 0.84
Esophagus 5 0.6
Head and neck 22 0.5909090909090909
Kidney 64 0.9375
Liver 25 1.0
Lung 53 0.9056603773584906
Muscle 1 0.0
Nerve 1 1.0
Pancreas 2 0.0
Prostate 26 0.5384615384615384
Skin 0 0
Stomach 16 0.5
Thymus 1 0.0
Thyroid 29 0.8620689655172413
Uterus 17 0.7058823529411765
Test Acc: 0.7967
